In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

In [4]:
k = 1000
split_pair_path = "../data/split_600w_2epoch/split_pair_dev_k_600w_2epoch_{0}.txt".format(k)
# split_pair_path = "../data/split_dev/split_pair_dev_k_{0}_all.txt".format(k)
# train_data_path = "../data/split_dev_encode_600w/train_set.txt"
# dev_data_path = "../data/split_dev_encode_600w/dev_set_v1_4.txt"

# train_data = pd.read_csv(train_data_path, sep='\t', header=None)
# train_label = np.array(train_data.iloc[:,0]).astype('int')
# dev_data = pd.read_csv(dev_data_path, sep='\t', header=None)
# dev_label = np.array(dev_data.iloc[:,0]).astype('int')
split_data = pd.read_csv(split_pair_path, sep='\t')

In [5]:
%%time
total_len = len(split_data)
unique_len = int(total_len/k)
original_shape = (unique_len, k)
true_sentence = np.array(split_data.true_sentence).reshape(original_shape)
candidate_sentence = np.array(split_data.candidate_sentence).reshape(original_shape)
pair_result = np.array(split_data.pair_result).reshape(original_shape)
true_label = np.array(split_data.true_label).reshape(original_shape)
candidate_label = np.array(split_data.candidate_label).reshape(original_shape)

CPU times: user 1.52 s, sys: 143 ms, total: 1.67 s
Wall time: 1.33 s


In [13]:
top = 100
np.mean(np.max(pair_result[:,:top], axis=1))

0.49361702127659574

In [30]:
new_train_data = split_data

In [32]:
# from probability file, get top k candidates
probability_file_path = "../data/first_filter_2epoch/test_results_k_{0}_of_k_1000.csv".format(k)

probability_file = pd.read_csv(probability_file_path, sep='\t',header=None).iloc[:,1]
probability_mat = np.array(probability_file).reshape(original_shape)

In [33]:
%%time
sorted_candidate_label = np.zeros((original_shape)).astype('int')
sorted_probability_mat = np.zeros((original_shape))
for i in range(unique_len):
    sorted_index = probability_mat[i].argsort()[::-1]
    sorted_candidate_label[i] = candidate_label[i, sorted_index]
    sorted_probability_mat[i] = probability_mat[i, sorted_index] 

CPU times: user 88.1 ms, sys: 288 µs, total: 88.4 ms
Wall time: 90.1 ms


In [34]:
def getFilteredLabel(weights, label_list):
    unique_label = np.unique(label_list)
    final_weights = np.zeros((unique_label.shape))
    for i, label in enumerate(unique_label):
        label_index = (label == label_list)
        final_weights[i] = sum(weights[label_index])
    sorted_index = final_weights.argsort()[::-1]
    return unique_label[sorted_index[0]]

def getWeightedLabel(weights, labels, threshold, filter=True):
    if filter:
        if weights[0] <= threshold:
            return labels[0]
        filtered_index = (weights > threshold)
        return getFilteredLabel(weights[filtered_index], labels[filtered_index])
    else:
        return getFilteredLabel(weights, labels)

def computeAccuracy(predicted, true_label):
    accuracy = accuracy_score(true_label, predicted)
    print("accuracy: {0}".format(accuracy))

In [42]:
%%time
top = 20
probability_threshold = 0.5
new_candidate_list = np.zeros((unique_len, 1)).astype('int')
for i in range(unique_len):
    new_candidate_list[i] = \
            getWeightedLabel(
                    sorted_probability_mat[i,:top],
                    sorted_candidate_label[i,:top],
                    probability_threshold, False)
computeAccuracy(new_candidate_list, true_label[:,0])

accuracy: 0.6945309026233881
CPU times: user 620 ms, sys: 4.21 ms, total: 625 ms
Wall time: 620 ms
